In [58]:
from collections import namedtuple

import mxnet as mx
from mxnet import gluon
import numpy as np

import dgl
import dgl.function as fn
from dgl.data import load_data

# Load a small dataset

cora is a small graph of 2708 nodes and 10556 edges.

In [15]:
DataArgs = namedtuple('DataArgs', 'dataset')
args = DataArgs('cora')
data = load_data(args)

print("""----Data statistics------'
      #Nodes %d
      #Edges %d
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d""" %
          (data.graph.number_of_nodes(),
           data.graph.number_of_edges(),
           data.num_labels,
           data.train_mask.sum(),
           data.val_mask.sum(),
           data.test_mask.sum()))

----Data statistics------'
      #Nodes 2708
      #Edges 10556
      #Classes 7
      #Train samples 140
      #Val samples 300
      #Test samples 1000


# Load data into DGL

In [50]:
g = dgl.DGLGraph(data.graph, readonly=True)
g.ndata['features'] = mx.nd.array(data.features)
g.ndata['labels'] = mx.nd.array(data.labels)

train_mask = mx.nd.array(data.train_mask)
val_mask = mx.nd.array(data.val_mask)
test_mask = mx.nd.array(data.test_mask)

train_nid = mx.nd.array(np.nonzero(data.train_mask)[0]).astype(np.int64)
test_nid = mx.nd.array(np.nonzero(data.test_mask)[0]).astype(np.int64)

[0.7399375  1.6960521  3.4994125  ... 0.36562112 3.2536626  0.61456096]


# Compute a node embedding with  2-layer GCN on a subset of nodes

To enable mini-batch training, we need to compute the node embeddings on a subset of nodes. For a 2-layer GCN, we first compute the node embeddings of the neighbors in the first layer of the GCN model.

$$h_i^{(1)} = \sigma(\Sigma_{j \in N(i)} h_j^{(0)}), \forall i \in N(v)$$

Then we compute the node embeddings of the second layer, which is the final embeddings of the nodes in this model.

$$h_v^{(2)} = \sigma(\Sigma_{i \in N(v)} h_i^{(1)})$$

The data and computation dependency is illustrated in the figure below:
![title](https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/Dependency.png)

In [78]:
class NodeUpdate(gluon.Block):
    def __init__(self, in_feats, out_feats, activation=None):
        super(NodeUpdate, self).__init__()
        self.dense = gluon.nn.Dense(out_feats, in_units=in_feats)
        self.activation = activation

    def forward(self, node):
        h = node.data['h']
        h = self.dense(h)
        if self.activation:
            h = self.activation(h)
        return {'h': h}

In [82]:
# Input dimension size
in_feats = data.features.shape[1]
# Hidden dimension size
n_hidden = 64

# If we want to compute the embeddings of the nodes 0, 1, 2
v = [0, 1, 2]

nfunc_layer1 = NodeUpdate(in_feats, n_hidden, mx.nd.relu)
nfunc_layer2 = NodeUpdate(n_hidden, n_hidden, mx.nd.relu)
nfunc_layer1.initialize()
nfunc_layer2.initialize()

# We first find the neighbors of these nodes.
neighbors = np.unique(g.in_edges(v)[0].asnumpy())
# compute the embeddings of the neighbors
g.pull(neighbors, fn.copy_src('features', 'msg'), fn.sum('msg', 'h'), nfunc_layer1)
# compute the embeddings of the nodes in the batch.
g.pull(v, fn.copy_src('h', 'msg'), fn.sum('msg', 'h'), nfunc_layer2)

The computation above is very expensive because computing the node embeddings of the model involve in many neighbor nodes. One solution is to use neighbor sampling to prune some of the dependency.

![title](https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/neighbor_sampling.png)

Neighbor sampling requires to perform pruning in every neighborhood separately. We could implement neighbor sampling with Python as follow. However, the code below is tedious and also very slow when running on a large graph.

In [84]:
# If we want to compute the embeddings of the nodes 0, 1, 2
nodes = [0, 1, 2]
num_neighs = 4

# Sample neighbors in each neighborhood separately.
for n in nodes:
    src2, dst2 = g.in_edges(n)
    idx = np.random.choice(np.arange(len(src2)), size=num_neighs)
    src2 = src2[idx]
    dst2 = dst2[idx]
    
    for neighbor in src2:
        src1, dst1 = g.in_edges(neighbor)
        idx = np.random.choice(np.arange(len(src1)), size=num_neighs)
        src1 = src1[idx]
        dst1 = dst1[idx]
        g.send_and_recv((src1, dst1), fn.copy_src('features', 'msg'), fn.sum('msg', 'h'), nfunc_layer1)
        
    g.send_and_recv((src2, dst2), fn.copy_src('h', 'msg'), fn.sum('msg', 'h'), nfunc_layer2)

# Create a mini-batch (NodeFlow) with neighbor sampling

Instead, DGL provides a set of sampling API for various sampling strategies. These sampling APIs create NodeFlows for mini-batch training on a graph.

The code below creates a neighbor sampler that creates a NodeFlow with 3 nodes in a batch and samples at maximal 4 neighbors on each node.

In [69]:
sampler = iter(dgl.contrib.sampling.NeighborSampler(g, 3, num_neighs, neighbor_type='in', num_hops=2))
nf = next(sampler)
nf.layer_parent_nid(-1)


[0 1 2]
<NDArray 3 @cpu(0)>

### Have a look at the data inside the NodeFlow.

The NodeFlow should have 3 layers and 2 blocks.

In [38]:
nf.num_layers

3

In [39]:
nf.num_blocks

2

Nodes in NodeFlow can be identified by two sets of Ids: global node Id (the node Ids in the original graph) and local node Id (used inside this NodeFlow). Local node Ids inside a NodeFlow are labelled starting from 0.

In [40]:
print(nf.layer_nid(0).asnumpy())
print(nf.layer_parent_nid(0).asnumpy())

[0 1 2 3 4 5 6 7 8 9]
[   0  305  333  348  384  565  613  751 1244 1440]


The NodeFlow should contain 10 nodes to compute node embeddings.

In [41]:
nodes = nf.layer_parent_nid(-1).asnumpy()
print("The sampled nodes in the batch: ", nodes)

The sampled nodes in the batch:  [0 1 2]


In [42]:
layer1_nodes = nf.layer_parent_nid(1).asnumpy()
full_graph_neighbors = g.in_edges(nodes)[0].asnumpy()
print("The nodes in the second layer: ", layer1_nodes)
print("The in-neighbors of the nodes: ", np.sort(full_graph_neighbors))

The nodes in the second layer:  [ 14 344 410 435 565]
The in-neighbors of the nodes:  [  8  14 258 344 410 435 471 544 552 565]


### Get data from the parent graph

In [45]:
try:
    print(nf.layers[0].data['features'])
except KeyError:
    print("features don't exist in node data of the NodeFlow")

features don't exist in node data of the NodeFlow


In [53]:
nf.copy_from_parent()
print(nf.layers[-1].data['labels'].asnumpy())
print(g.nodes[nf.layer_parent_nid(-1)].data['labels'].asnumpy())

[2. 5. 4.]
[2. 5. 4.]


### Trigger the computation in the graph

In [56]:
nf.layers[0].data['h'] = nf.layers[0].data['features']
for i in range(nf.num_blocks):
    nf.block_compute(i, fn.copy_src('h', 'msg'), fn.sum('msg', 'h'))

/Users/dzzhen/Workspace/dgl/python/dgl/base.py:18: UserWarning: Initializer is not set. Use zero initializer instead. To suppress this warning, use `set_initializer` to explicitly specify which initializer to use.
  warnings.warn(msg)


# Train GCN with neighbor sampling
In an $L$-layer graph convolution network (GCN), given a graph $G=(V, E)$, represented as an adjacency matrix $A$, with node features $H^{(0)} = X \in \mathbb{R}^{|V| \times d}$, the hidden feature of a node $v$ in $(l+1)$-th layer $h_v^{(l+1)}$ depends on the features of all its neighbors in the previous layer $h_u^{(l)}$:
$$
z_v^{(l+1)} = \sum_{u \in \mathcal{N}(v)} \tilde{A}_{uv} h_u^{(l)} \qquad h_v^{(l+1)} = \sigma ( z_v^{(l+1)} W^{(l)})
$$
where $\mathcal{N}(v)$ is the neighborhood of $v$, $\tilde{A}$ could be any normalized version of $A$ such as $D^{-1} A$ in Kipf et al., $\sigma(\cdot)$ is an activation function, and $W^{(l)}$ is a trainable parameter of the $l$-th layer.

The node update function is still the same as before.

In [61]:
class NodeUpdate(gluon.Block):
    def __init__(self, in_feats, out_feats, activation=None):
        super(NodeUpdate, self).__init__()
        self.dense = gluon.nn.Dense(out_feats, in_units=in_feats)
        self.activation = activation

    def forward(self, node):
        h = node.data['h']
        h = self.dense(h)
        if self.activation:
            h = self.activation(h)
        return {'activation': h}

Instead of using all the $L$-hop neighbors of a node $v$, [Hamilton et al.](https://arxiv.org/abs/1706.02216) propose *neighbor sampling*, which randomly samples a few neighbors $\hat{\mathcal{N}}^{(l)}(v)$ to estimate the aggregation $z_v^{(l+1)}$ of its total neighbors $\mathcal{N}(v)$ in $l$-th GCN layer, by an unbiased estimator $\hat{z}_v^{(l+1)}$
$$
\hat{z}_v^{(l+1)} = \frac{\vert \mathcal{N}(v) \vert }{\vert \hat{\mathcal{N}}^{(l)}(v) \vert} \sum_{u \in \hat{\mathcal{N}}^{(l)}(v)} \tilde{A}_{uv} \hat{h}_u^{(l)} \qquad
\hat{h}_v^{(l+1)} = \sigma ( \hat{z}_v^{(l+1)} W^{(l)} )
$$
Let $D^{(l)}$ be the number of neighbors to be sampled for each node at the $l$-th layer,
then the receptive field size of each node can be controlled under $\prod_{i=0}^{L-1} D^{(l)}$ by *neighbor sampling*.

Here we define the node UDF which is a fully-connected layer:

In [62]:
class GCNSampling(gluon.Block):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout,
                 **kwargs):
        super(GCNSampling, self).__init__(**kwargs)
        self.dropout = dropout
        self.n_layers = n_layers
        with self.name_scope():
            self.layers = gluon.nn.Sequential()
            # input layer
            self.layers.add(NodeUpdate(in_feats, n_hidden, activation))
            # hidden layers
            for i in range(1, n_layers-1):
                self.layers.add(NodeUpdate(n_hidden, n_hidden, activation))
            # output layer
            self.layers.add(NodeUpdate(n_hidden, n_classes))

    def forward(self, nf):
        nf.layers[0].data['activation'] = nf.layers[0].data['features']
        for i, layer in enumerate(self.layers):
            h = nf.layers[i].data.pop('activation')
            if self.dropout:
                h = mx.nd.Dropout(h, p=self.dropout)
            nf.layers[i].data['h'] = h
            # block_compute() computes the feature of layer i given layer
            # i-1, with the given message, reduce, and apply functions.
            # Here, we essentially aggregate the neighbor node features in
            # the previous layer, and update it with the `layer` function.
            nf.block_compute(i,
                             fn.copy_src(src='h', out='m'),
                             lambda node : {'h': node.mailbox['m'].mean(axis=1)},
                             layer)
        h = nf.layers[-1].data.pop('activation')
        return h

In [67]:
# dropout probability
dropout = 0.2
# batch size
batch_size = 100
# number of neighbors to sample
num_neighbors = 4
# number of epochs
num_epochs = 30

# The number of classes we want to classify the nodes
n_classes = data.num_labels
# The number of layers of GCN.
L = 2

# initialize the model and cross entropy loss
model = GCNSampling(in_feats, n_hidden, n_classes, L,
                    mx.nd.relu, dropout, prefix='GCN')
model.initialize()
loss_fcn = gluon.loss.SoftmaxCELoss()

# use adam optimizer
trainer = gluon.Trainer(model.collect_params(), 'adam',
                        {'learning_rate': 0.03, 'wd': 0})

for epoch in range(num_epochs):
    for nf in dgl.contrib.sampling.NeighborSampler(g, batch_size,
                                                   num_neighbors,
                                                   neighbor_type='in',
                                                   shuffle=True,
                                                   num_hops=L,
                                                   seed_nodes=train_nid):
        # When `NodeFlow` is generated from `NeighborSampler`, it only contains
        # the topology structure, on which there is no data attached.
        # Users need to call `copy_from_parent` to copy specific data,
        # such as input node features, from the original graph.
        nf.copy_from_parent()
        with mx.autograd.record():
            # forward
            pred = model(nf)
            batch_nids = nf.layer_parent_nid(-1).astype('int64')
            batch_labels = labels[batch_nids]
            # cross entropy loss
            loss = loss_fcn(pred, batch_labels)
            loss = loss.sum() / len(batch_nids)
        # backward
        loss.backward()
        # optimization
        trainer.step(batch_size=1)
        print("Epoch[{}]: loss {}".format(epoch, loss.asscalar()))

Error in autograd.Function.delete: Traceback (most recent call last):
  File "/Users/dzzhen/Workspace/incubator-mxnet/python/mxnet/autograd.py", line 482, in delete_entry
    del Function._registry.ref_holder[key]
KeyError: 844

Epoch[0]: loss 1.9462796449661255
Error in autograd.Function.delete: Traceback (most recent call last):
  File "/Users/dzzhen/Workspace/incubator-mxnet/python/mxnet/autograd.py", line 482, in delete_entry
    del Function._registry.ref_holder[key]
KeyError: 846

Epoch[0]: loss 1.9235894680023193
Error in autograd.Function.delete: Traceback (most recent call last):
  File "/Users/dzzhen/Workspace/incubator-mxnet/python/mxnet/autograd.py", line 482, in delete_entry
    del Function._registry.ref_holder[key]
KeyError: 848

Epoch[1]: loss 1.8674328327178955
Error in autograd.Function.delete: Traceback (most recent call last):
  File "/Users/dzzhen/Workspace/incubator-mxnet/python/mxnet/autograd.py", line 482, in delete_entry
    del Function._registry.ref_holder[ke

Epoch[16]: loss 0.34984031319618225
Error in autograd.Function.delete: Traceback (most recent call last):
  File "/Users/dzzhen/Workspace/incubator-mxnet/python/mxnet/autograd.py", line 482, in delete_entry
    del Function._registry.ref_holder[key]
KeyError: 912

Epoch[17]: loss 0.5049612522125244
Error in autograd.Function.delete: Traceback (most recent call last):
  File "/Users/dzzhen/Workspace/incubator-mxnet/python/mxnet/autograd.py", line 482, in delete_entry
    del Function._registry.ref_holder[key]
KeyError: 914

Epoch[17]: loss 0.23732736706733704
Error in autograd.Function.delete: Traceback (most recent call last):
  File "/Users/dzzhen/Workspace/incubator-mxnet/python/mxnet/autograd.py", line 482, in delete_entry
    del Function._registry.ref_holder[key]
KeyError: 916

Epoch[18]: loss 0.39064860343933105
Error in autograd.Function.delete: Traceback (most recent call last):
  File "/Users/dzzhen/Workspace/incubator-mxnet/python/mxnet/autograd.py", line 482, in delete_entry